In [ ]:
# | default_exp cli

# Command Line Interface

In [ ]:
# | hide
from fastcore.test import *
import jupyter_black
import tempfile

jupyter_black.load()

In [ ]:
# | export
import click
from pathlib import Path
from sal.core import Data
from sal.loaders import xml_file_to_data
from sal.templates import MissingTemplate
from IPython import get_ipython
from sal.codegen import (
    Sal,
    FrontMatterInMemoryTemplateLoader,
    Renderer,
    JinjaTemplateRenderer,
)

from typing import Any

## Generating code


Up until now, we've been developing the code we need to generate code. Now it's time to wrap that code in a easy to use function to use as a command line interface. This cli will mirror the args of this function so:

- it accepts an xml file path as input
- it accepts a template directory as an input

Also, when a certain template does not exist, it will create one with a default template `Renderer.DEFAULT_TEMPLATE`.

In [ ]:
# | export
def _render(file: str, directory: str) -> str | Any:
    try:
        repository = FrontMatterInMemoryTemplateLoader.from_directory(directory)
        renderer = Renderer(repository=repository, renderer=JinjaTemplateRenderer())
        sal = Sal(renderer)

        struct: Data = xml_file_to_data(file)
        return sal.process(struct)
    except MissingTemplate as e:
        path = Path(directory) / f"{e.name}.jinja2"
        path.write_text(Renderer.DEFAULT_TEMPLATE)
        return render(file, directory)

In [ ]:
# | hide
model = """
---
reference:  "sigla-{{ node.attrs.name | lower }}-model"
---
class {{ name }}Model(models.Model): # {{ reference }}
    {% for child in children -%}
    {{ child | render }}
    {% endfor %}
"""

field = """
---
reference:  "sigla-{{ node.name | lower }}-model"
---
{{ name }} = models.{{ type | title }}Field() 
"""


destination = tempfile.NamedTemporaryFile()

xml = f"""
<wrapper>
    <to-file to="{destination.name}">
        <model name="User">
            <field name="id" type="integer"/>
            <field name="username" type="char"/>
            <field name="email" type="email"/>
        </model>
    </to-file>
</wrapper>
"""

In [ ]:
# | hide
# prepare files and directories

Path(destination.name).unlink(missing_ok=True)

# create template dir
path = Path("/tmp/templates")
path.mkdir(exist_ok=True)

# create xml file
Path("/tmp/sal.xml").write_text(xml)

# create template files
Path("/tmp/templates/model.jinja2").write_text(model)
Path("/tmp/templates/field.jinja2").write_text(field)

106

In [ ]:
# render
_render("/tmp/sal.xml", "/tmp/templates")

result = Path(destination.name).read_text().strip()

test_eq(
    result,
    """
class UserModel(models.Model): # sigla-user-model
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
""".strip(),
)

---

In [ ]:
# | export
@click.group()
def main() -> None:
    pass

In [ ]:
# | export

# TODO : init command
# - create : sal.xml file
# - create : sal folder
# - create : sal/templates folder


@main.command()
@click.option("--filename", type=click.Path(exists=True), default="./sal.xml")
@click.option("--folder", type=click.Path(exists=True), default="./sal")
def render(filename: str, folder: str) -> None:
    click.echo(f"⚠️ {filename=}")
    click.echo(f"⚠️ {folder=}")
    _render(filename, str(Path(folder) / "templates"))

In [ ]:
# | export
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == "ZMQInteractiveShell" or shell == "CaptureShell":
            return True  # Jupyter notebook or qtconsole
        elif shell == "TerminalInteractiveShell":
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False  # Probably standard Python interpreter

In [ ]:
# | export
if __name__ == "__main__" and not is_notebook():
    main()

---

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()